# Neural Data

Check and explore the neural data for a single-unit project,

This notebook covers loading data, preparing metadata, getting micro-LFP data, and splitting channels ready for sorting.

In [ ]:
import h5py
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# EXTRA STUFF FOR DEALING WITH MICRO LFPS
#import scipy.io as sio
#from scipy.signal import butter, resample, filtfilt
#from scipy.fftpack import next_fast_len
#import joblib

In [ ]:
# Import convnwb functions
from convnwb.io import get_files, load_config, save_json, load_json
from convnwb.paths import Paths

In [ ]:
# Add brPY to path
import sys
br_path = '/home1/tom.donoghue/tools/brPY'
sys.path.append(br_path)
from brpylib import NsxFile

## Settings

In [ ]:
# Set the project path
project_path = '/data12/jacobs_lab/BAU01'

In [ ]:
# Set session information
SUBJ = {
    'subject' : 'UT202014',
    'experiment' : 'T3', 
    'session' : 0,
}

In [ ]:
# Define which site the subject is from
#site = 'Baylor'
site = 'Utah'

In [ ]:
# Create database structure for the current file
data_paths = Paths(project_path, **SUBJ)

## Get & Check Files

In [ ]:
# Get the list of available neural data files
data_paths.get_files('neural')

In [ ]:
# Check available ns5 files
data_files = data_paths.get_files('neural', select='ns5')
data_files

In [ ]:
# Select data file to process
data_file = data_files[0]
data_file

## Load Data Files

Notes (to finish):
- generalizing to ns5 or ns6 files
- working if there is more than one NSP data file
- Baylor: selecting the correct sheet from the excel sheet

In [ ]:
# Open the Nsx file 
nsxfile = NsxFile(str(data_paths.neural / data_file))

In [ ]:
# Get the data 
nsp_data = nsxfile.getdata()
nsxfile.close()

In [ ]:
# Pull out the the ephys data, then rename remaining info
data = nsp_data.pop('data')
params = nsp_data

In [ ]:
# Check data shape
print(data.shape)

### Check NSP information



In [ ]:
# Check params information
params

In [ ]:
len(params['elec_ids'])

In [ ]:
# # Write out metadata parameters
# save_json(params, 'params.json', folder=db.sorting)

In [ ]:
# Reload params info
#params = load_json('params', folder=db.sorting)

### Audio Sync Pulses

Audio sync channels: typically the NSP 1 file, with 11 channels, has audio sync pulses on 1th channel

In [ ]:
_, ax = plt.subplots(figsize=(12, 4))
plt.plot(data[1, 0:10000000])
#plt.plot(data[1, 20000000:30000000])
#plt.plot(data[1, :])

In [ ]:
# Define name for the audio sync file
audio_sync_name = 'audio_sync_pulses.hdf5'

In [ ]:
# # Save out audio channel
# file = h5py.File(data_paths.alignment / audio_sync_name, "w")
# file.create_dataset('data', data=data[1, :], dtype='<i2')
# file.close()

## Split Channel Files

In [ ]:
# Check electrode IDs that are in the loaded params file and in the electrodes sheet
matching_eids = [eid for eid in params['elec_ids'] if eid in electrodes.channel_index.values]
matching_eids

In [ ]:
# Split the data by channels and write into hdf5 files
print('\n\nSPLITTING CHANNEL FILES\n\n')
for ix, electrode in enumerate(params['elec_ids']):

    if electrode in electrodes.channel_index:
        
        print('  splitting electrode file: {}'.format(electrode))

#         # Save out HDF5 of microdata        
#         file = h5py.File(db.split_files / 'chan_{}.hdf5'.format(electrode), "w")
#         file.create_dataset('data', data=data[ix, :], dtype='<i2')
#         file.close()

In [ ]:
# STUFF FOR ABOVE LOOP FOR SAVING OUT MICRO LFP TRACES

#         ## Pre-process and save out the micro-wire LFP
#         rfs = 500
#         fc = 250
#         filt_ord = 4

#         # Do a lowpass filter (250 Hz) on the LFP data
#         [b_vals, a_vals] = butter(filt_ord, fc / (fs / 2), 'low')
#         filted = filtfilt(b_vals, a_vals, data[ix, :])

#         # Then downsample it to 500 Hz 
#         pad = np.ones([next_fast_len(len(filted)) - len(filted)])
#         lfp = resample(np.append(filted, pad), int(len(np.append(filted, pad)) / (fs / rfs)))
#         lfp = lfp[:-int(len(pad) / (fs / rfs))]

#         # TODO: This might change soon! 
#         joblib.dump(lfp, f'{directories['micro_lfp']}/chan{electrode}.p')

In [ ]:
# Using the blackrock code
# nsxfile.savesubsetnsx(elec_ids=[electrode], file_suffix='chan_{}'.format(electrode))
# Matfile: This is redundant with HDF5 
# data_as_dict = {'data': data[ix, :].astype('double'), 'fs':30000}
# sio.savemat("{}_chan{}.mat".format(path, electrode), data_as_dict)

## MISC EXTRA

In [ ]:
# # UT registered data: 
# # channel_map_file = '/home1/salman.qasim/T3_Data/UT202014/Registered/ChannelMap.mat'
# electrode_file = '/home1/salman.qasim/T3_Data/UT202014/Registered/Electrodes.mat'
# f = h5py.File(electrode_file, 'r')

# np.array(f['ElecXYZMNIRaw'])[0, :].shape

# pd.read_csv('/home1/salman.qasim/T3_Data/UT202014/UT202014labels.csv')